In [2]:
import sys, os
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(r"C:\Users\super\DAG")
sys.path.append(r"c:\Users\super\DAG\golemMain\src")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from MEC import is_in_markov_equiv_class
print(os.getcwd())
from SCM_data import generate_scm_from_BN 
from numpy.linalg import inv
from scipy.linalg import sqrtm
from nodag.nodag_gumbel_softmax import train_gumbel_sgd_dag
from nodag.nodag_gs_findbest import nodag_fb_l_dag
from golemMain.src.golem import golem
from coordinate_descent.coordinate import dag_coordinate_descent, weight_to_adjacency
from coordinate_descent.coordinate0 import dag_coordinate_descent_l0

c:\Users\super\DAG\experiments

c:\Users\super\DAG\experiments
c:\Users\super\DAG\experiments


In [3]:
experiments = []

# ----------- Experiment 1 -----------
experiments.append({
    "name": "d=3, A→B←C",
    "B_true": np.array([
        [0, 1, 0],
        [0, 0, 0],
        [0, 2, 0]
    ]),
    "N": np.array([1, 2, 3]),
    "Omega_true": np.diag([1, 2, 3])
})

# ----------- Experiment 2 -----------
experiments.append({
    "name": "d=3, A→B→C",
    "B_true": np.array([
        [0, 1, 0],
        [0, 0, 3],
        [0, 0, 0]
    ]),
    "N": np.array([1, 3, 4]),
    "Omega_true": np.diag([1, 3, 4])
})

# ----------- Experiment 3 -----------
experiments.append({
    "name": "d=3, A→B→C + A→C",
    "B_true": np.array([
        [0, 1, 2],
        [0, 0, 3],
        [0, 0, 0]
    ]),
    "N": np.array([5, 4, 3]),
    "Omega_true": np.diag([5, 4, 3])
})

# ----------- Experiment 4 -----------
experiments.append({
    "name": "d=4, A→B, B→C, B→D",
    "B_true": np.array([
        [0, 3, 0, 0],
        [0, 0, 3, 4],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    "N": np.array([1, 3, 3, 2]),
    "Omega_true": np.diag([1, 3, 3, 2])
})

# ----------- Experiment 5 -----------
experiments.append({
    "name": "d=4, A→C, A→D, B→C, B→D",
    "B_true": np.array([
        [0, 0, 2, 3],
        [0, 0, 3, 4],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    "N": np.array([2, 4, 3, 5]),
    "Omega_true": np.diag([2, 4, 3, 5])
})

# ----------- Experiment 6 -----------
experiments.append({
    "name": "d=4, A→D, B→D, C→D",
    "B_true": np.array([
        [0, 0, 0, 1],
        [0, 0, 0, 3],
        [0, 0, 0, 5],
        [0, 0, 0, 0]
    ]),
    "N": np.array([5, 4, 3, 2]),
    "Omega_true": np.diag([5, 4, 3, 2])
})

# ----------- Experiment 7 -----------
experiments.append({
    "name": "d=5, e=4, |v|=0",
    "B_true": np.array([
        [0, 1, 0, 2, 0],
        [0, 0, 3, 0, 4],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})

# ----------- Experiment 8 -----------
experiments.append({
    "name": "d=5, e=4, |v|=1",
    "B_true": np.array([
        [0, 0, 1, 2, 0],
        [0, 0, 0, 2, 3],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})

# ----------- Experiment 9 -----------
experiments.append({
    "name": "d=5, e=4, |v|=2",
    "B_true": np.array([
        [0, 0, 0, 1, 0],
        [0, 0, 0, 2, 3],
        [0, 0, 0, 0, 4],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})


In [4]:
def run_one_coordinate0(B_true, N, name=None,
                       n_samples=10000,
                       lambda_l0 = 0.0,
                       threshold=0.05,
                       T=300,
                       n_repeats=100,
                       seed_data = 42):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=seed_data
    )
    R_hat = np.cov(data.T)
    # print("R_hat =\n", np.array2string(
    #     R_hat, formatter={'float_kind': lambda x: f"{x:.2f}"}
    # ))

    d = R_hat.shape[0]

    Omega_true = np.diag(N)
    # print("inv(sqrtm(Omega_true)) =\n", inv(sqrtm(Omega_true)))

    Theta_hat = inv(R_hat)
    A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
    # print("A_true =\n", np.array2string(
    #     A_true, formatter={'float_kind': lambda x: f"{x:.2f}"}
    # ))

    likelihood_true = -2 * np.log(np.linalg.det(A_true)) \
                      + np.trace(A_true.T @ R_hat @ A_true)
    # print("likelihood_true =", likelihood_true)

    correct = 0
    for seed in range(n_repeats):
        A_opt, G_opt, fval = dag_coordinate_descent_l0(
            R_hat, T=T, seed=seed, threshold=threshold, lambda_l0=lambda_l0
        )
        if is_in_markov_equiv_class(G_true, G_opt):
            correct += 1

    correct_rate = correct / n_repeats
    print("Correct rate =", correct_rate)
    return correct_rate


In [5]:
# N = 10000
# T = 5000
# lambda = 0.0

print("N_samples = 10000")
correct_rate = []
for seed_data in range(10):
    print("seed of data: ",seed_data)
    for exp in experiments:
        correct_rate.append(
            run_one_coordinate0(
                B_true=exp["B_true"],
                N=exp["N"],
                name=exp["name"],
                n_samples=10000,
                lambda_l0=0.0,
                threshold=0.05,
                T=5000,
                n_repeats=100,
                seed_data=seed_data
            )
        )



N_samples = 10000
seed of data:  0
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.82
Running: d=3, A→B→C + A→C
Correct rate = 1.0
Running: d=4, A→B, B→C, B→D
Correct rate = 0.69
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.62
Running: d=4, A→D, B→D, C→D
Correct rate = 0.82
Running: d=5, e=4, |v|=0
Correct rate = 0.41
Running: d=5, e=4, |v|=1
Correct rate = 0.78
Running: d=5, e=4, |v|=2
DAG/k constraint, continue
Correct rate = 0.54
seed of data:  1
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.82
Running: d=3, A→B→C + A→C
Correct rate = 1.0
Running: d=4, A→B, B→C, B→D
Correct rate = 0.7
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.62
Running: d=4, A→D, B→D, C→D
Correct rate = 0.82
Running: d=5, e=4, |v|=0
Correct rate = 0.39
Running: d=5, e=4, |v|=1
DAG/k constraint, continue
Correct rate = 0.58
Running: d=5, e=4, |v|=2
DAG/k constraint, continue
DAG/k constraint, continue
Correct rate = 0.57
seed of data:  2
Runnin